## Parameters For Grid Valuation

### Pricing For Cloud Capacity As Used In Cloud Index

See cloudorado for market price comparison [storage](https://www.cloudorado.com/cloud_storage_comparison.jsp) and [compute](https://www.cloudorado.com/cloud_server_comparison.jsp)

- For SU = 1 TB of storage per month
    - most cloud vendors use USD20-190 per month
- For CU = 4GB Memory and 2 virtual CPU
    - most cloud vendors use USD40-200 per month
    
We suggest to use 

- 15 USD for CU
- 10 USD for SU

### role of price erosion and capabilities increase of hardware

Over time the price for capacity goes down with a certain percentage

- we call this percentage price_erosion
- the price erosion has been defined in params and is default 40%

Also hardware becomes more powerful and over time can produce more CPR

CPR is the cloud production ratio, which is defined as ```1.5*CU+1*SU```

- we call this percentage cpr_improve and is default also 40%

In other words price erosion +- offsets the increased capabilities of hardware.






In [1]:
import ipywidgets as w
from Jumpscale import j
debounce = j.servers.notebook.debounce
config = j.tools.tfgrid_simulator.simulator_config.cloudindex

#define some layout
items_layout = w.Layout( width='auto')
box_layout = w.Layout(display='flex',flex_flow='column', align_items='stretch',width='50%')

# the input box
label_prices = w.Label("Index Cloud Unit Prices")
price_cu=w.IntSlider(description="CU Price",layout=items_layout,min=8,max=200,step=1,value=config.price_cu)
price_su=w.IntSlider(description="SU Price",layout=items_layout,min=5,max=120,step=1,value=config.price_su)
price_nu=w.FloatSlider(description="NU Price",layout=items_layout,min=0.00,max=1,step=0.01,value=config.price_nu)

label_simulationchoice = w.Label("Cloud Valuation Type:")
simulation_type = w.Dropdown(
    options=["revenue","margin"],
    value=config.indextype,
    description='Type:',
    disabled=False,
layout=w.Layout( width='85%')
)



multiplier_revenue=w.IntSlider(description="Multiplier",layout=items_layout,min=12,max=200,step=1,
                        value=config.revenue_months)
multiplier_margin=w.IntSlider(description="Multiplier",layout=items_layout,min=12,max=200,step=1,
                        value=config.margin_months)

if config.indextype=="revenue":
    multiplier=multiplier_revenue
else:
    multiplier=multiplier_margin

label_is_months = w.Label("is in months.",layout=w.Layout(align_items="center",width='90%'))
# label_is_months.align_items="center"
# label_is_months.layout.justify_content = "flex-end"
#TODO: want to align right, can't find it
    
box_layout_input = w.Layout(display='flex',flex_flow='column',
    align_items='stretch',width='100%')

box_left=w.HBox(children=[price_cu,price_su,price_nu,label_simulationchoice,
                          simulation_type,multiplier,label_is_months],layout=box_layout_input)

inputbox = w.HBox(children=[box_left])

button_run=w.Button(description="SAVE", layout=w.Layout(width='100px'), 
    button_style='danger')
# button_clear=w.Button(description="Clear", layout=w.Layout(width='100px'), 
#     button_style='danger')
buttons = w.HBox(children=[button_run])

#now add all under neith each other
mainbox= w.VBox(children=[label_prices,inputbox,buttons])

#will store it in redis
def save(*args):
    config.price_cu = price_cu.value
    config.price_su = price_su.value
    config.price_nu = price_nu.value
    
    indextype=str(simulation_type.value)
    print(simulation_type.value)
    config.indextype = indextype
    
    if indextype == "revenue":
        config.revenue_months = int(multiplier.value)
    else:
        config.margin_months = int(multiplier.value)
        
    #doesn't work, idea is to replace it
#     box_left.children[5]=multiplier #TODO:

    j.tools.tfgrid_simulator.simulator_config.tft_pricing_type="AUTO"

    j.tools.tfgrid_simulator.simulator_config.save()
    
button_run.on_click(save)

def simulation_type_change(change):
    print("simulation change:%s"%simulation_type.value)
    if simulation_type.value=="revenue":
        multiplier.value=value=config.revenue_months        
    else:
        multiplier.value=value=config.margin_months 

simulation_type.observe(simulation_type_change)
        
mainbox

In [2]:
j.tools.tfgrid_simulator.simulator_config.load()
j.tools.tfgrid_simulator.simulator_config;


## Some remarks

A baseline is being established of 4% growth for TFT per month, we do this to allow forward looking estimation of grid in beginning.

Once the calculated TFT price based on chosen index valuation mechanism is higher than the baseline we will switch over to that baseline.